<a href="https://colab.research.google.com/github/sfn5116/DS330/blob/master/DS320_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Data Integration (Part I: Extraction and Cleaning)

This is the first part of a two part exercise to integrate and compare two different movie datasets. One is a CSV file from Kaggle and the other is semi-structured text data from the Internet Movie Database (IMDB).

In this first part, we will extract information from all the datasets and load the data into a data structure called a Dataframe (part of the Pandas data science library in Python). We will then get rid of duplicates and begin to clean the data to make the data integration proceed more smoothly.


Look at  this first code block. It fetches files from the internet locations (the Kaggle dataset comes from Github and the IMDB data set (two files) come from an ftp site. It places the data files in a newly created directory called 'data' in your filespace. Before running this code block, **PLACE YOUR NAME AS A COMMENT AT THE TOP**.  Then, go ahead and run this code block. 

In [1]:
#Srikar Nimmalapudi

# Importing urlib
import urllib.request
import os

# Creating the data folder
if not os.path.exists('./data'):
	os.makedirs('./data')

# Obtaining the dataset using the url that hosts it
kaggle_url = 'https://github.com/sundeepblue/movie_rating_prediction/raw/master/movie_metadata.csv'
if not os.path.exists('./data/kaggle_dataset.csv'):     # avoid downloading if the file exists
	response = urllib.request.urlretrieve(kaggle_url, './data/kaggle_dataset.csv')
import gzip

# Obtaining IMDB's text files
imdb_url_prefix = 'ftp://ftp.funet.fi/pub/mirrors/ftp.imdb.com/pub/frozendata/'
imdb_files_list = ['genres.list.gz', 'ratings.list.gz']
for name in imdb_files_list:
	if not os.path.exists('./data/' + name):
		response = urllib.request.urlretrieve(imdb_url_prefix + name, './data/' + name)
		urllib.request.urlcleanup()   # urllib fails to download two files from a ftp source. This fixes the bug!
		with gzip.open('./data/' + name, 'rb') as comp_file, open('./data/' + name[:-3], 'wb') as reg_file:
			file_content = comp_file.read()
			reg_file.write(file_content)

In [2]:
imdb_url = 'https://anaconda.org/BigGorilla/datasets/1/download/imdb_dataset.csv'
if not os.path.exists('./data/imdb_dataset.csv'):     # avoid downloading if the file exists
	response = urllib.request.urlretrieve(kaggle_url, './data/imdb_dataset.csv')

**Data Extraction**

The IMDB Plain Text Data file is a collection of semi-structured text files that need to be processed to extract the data. We will be looking at the files "ratings.list" and "genres.list". A quick look at the first lines of each file will show that the files have a different format and we must extract the data separately. We will use regular expressions (RE) to extract the data from the file. In particular, we will use the group method in Python RE to extract the different data elements (e.g. title, year, etc.) into groups that will become columns in a Pandas dataframe


In [3]:
with open("./data/ratings.list", encoding='ISO-8859-1') as myfile:
	head = [next(myfile) for x in range(38)]
print (''.join(head[28:38]))   # skipping the first 28 lines as they are descriptive headers

      0000000125  1888533   9.2  The Shawshank Redemption (1994)
      0000000125  1289428   9.2  The Godfather (1972)
      0000000124  889607   9.0  The Godfather: Part II (1974)
      0000000124  1864164   9.0  The Dark Knight (2008)
      0000000133  518449   8.9  12 Angry Men (1957)
      0000000133  971107   8.9  Schindler's List (1993)
      0000000123  1477112   8.9  Pulp Fiction (1994)
      0000000124  1349449   8.9  The Lord of the Rings: The Return of the King (2003)
      0000000123  559468   8.8  Il buono, il brutto, il cattivo (1966)
      0000000133  1513600   8.8  Fight Club (1999)



In [4]:
with open("./data/genres.list") as myfile:
	head = [next(myfile) for x in range(392)]
print (''.join(head[382:392]))   # skipping the first 382 lines as they are descriptive header


"!Next?" (1994)						Documentary
"#1 Single" (2006)					Reality-TV
"#15SecondScare" (2015)					Horror
"#15SecondScare" (2015)					Short
"#15SecondScare" (2015)					Thriller
"#15SecondScare" (2015) {Who Wants to Play with the Rabbit? (#1.2)}	Drama
"#15SecondScare" (2015) {Who Wants to Play with the Rabbit? (#1.2)}	Horror
"#15SecondScare" (2015) {Who Wants to Play with the Rabbit? (#1.2)}	Short



**Extracting information from genres.list** The  goal  of  this  step  is  to  extract  the movie titles and their production year, and genre from “genres.list”, and store the extracted data into a dataframe.  Dataframe (from the python package pandas) is one of the key tools that is commonly used for data profiling and cleaning.  To extract the desired information from thetext, we rely on regular expressions which are implemented in the python package “re”.

In [5]:
import re
import pandas as pd

with open("./data/genres.list",encoding='ISO-8859-1') as genres_file:
	raw_content = genres_file.readlines()
	genres_list = []
	content = raw_content[382:]
	for line in content:
		m = re.match(r'"?(.*[^"])"? \(((?:\d|\?){4})(?:/\w*)?\).*\s((?:\w|-)+)', line.strip())
		if m is None: continue
		genres_list.append([m.group(1), m.group(2), m.group(3)])
	genres_data = pd.DataFrame(genres_list, columns=['movie', 'year', 'genre'])
	print (genres_data)   

                  movie  year        genre
0                !Next?  1994  Documentary
1             #1 Single  2006   Reality-TV
2        #15SecondScare  2015       Horror
3        #15SecondScare  2015        Short
4        #15SecondScare  2015     Thriller
...                 ...   ...          ...
2658936        überRICH  2017       Comedy
2658937        überRICH  2017        Short
2658938              üç  2012    Adventure
2658939              üç  2012       Comedy
2658940              üç  2012        Short

[2658941 rows x 3 columns]


Now do the same thing, extracting Movie, year, and rating into a Pandas Dataframe (IN THAT ORDER) from ratings.list. You can use the regular expression in this next code block by copying it:  

In [ ]:
#Only grab the regular expression from the line below
#m = re.match(r'(?:\d|\.|\*){10}\s+\d+\s+(1?\d\.\d)\s"?(.*[^"])"? \(((?:\d|\?){4})(?:/\w*)?\)', line.strip())
#
#Look at the code blocks above where we explore ratings.list and genres.list. We skip header lines in each one. We skip the first
#382 lines in genres.list and then when we go and create the Dataframe above, we do the same with the content= line. For ratings.list
#skip only the first 28 lines which have headers. 
 

You can probably mimic the code block above for genres, except on ratings.list. You will need different variables for ratings by changing variable names with "genres" to "ratings". This will put the data into RE groups. Note that the ?: operator in a group denotes an ignored group and that the group numbering will always proceed in the order retrieved from the file beginning with 1. Create your code block in the cell below: 


In [13]:
with open("./data/ratings.list",encoding='ISO-8859-1') as ratings_file:
	raw_content = ratings_file.readlines()
	ratings_list = []
	content = raw_content[28:]
	for line in content:
		m = re.match(r'(?:\d|\.|\*){10}\s+\d+\s+(1?\d\.\d)\s"?(.*[^"])"? \(((?:\d|\?){4})(?:/\w*)?\)', line.strip())
		if m is None: continue
		ratings_list.append([m.group(1), m.group(2), m.group(3)])
	ratings_data = pd.DataFrame(ratings_list, columns=['rating', 'movie', 'year'])
	print (ratings_data)  

       rating                      movie  year
0         9.2   The Shawshank Redemption  1994
1         9.2              The Godfather  1972
2         9.0     The Godfather: Part II  1974
3         9.0            The Dark Knight  2008
4         8.9               12 Angry Men  1957
...       ...                        ...   ...
789410    8.8                     à deux  2017
789411    7.0                     ärtico  2014
789412    5.4        éX-Driver the Movie  2002
789413    6.2                         él  2001
789414    6.8                          ö  2012

[789415 rows x 3 columns]


Since the Kaggle data already exists in a structured csv file, loading it into a Pandas Dataframe is straightforward:


In [8]:
import pandas as pd

# Loading the Kaggle dataset from the .csv file (kaggle_dataset.csv)
kaggle_data = pd.read_csv('./data/kaggle_dataset.csv')

Let's find out how many movies are in the Kaggle data set:

In [9]:
print ('Number of movies in kaggle_data: {}'.format(kaggle_data.shape[0]))


Number of movies in kaggle_data: 5043


Let's do the same for genres_data and ratings_data. Complete the code block below to print the number of movies in ratings_data and genres_data:


In [10]:
print ('Number of movies in genres_data: {}'.format(genres_data.shape[0]))
print ('Number of movies in ratings_data: {}'.format(ratings_data.shape[0]))

Number of movies in genres_data: 2658941
Number of movies in ratings_data: 789415


There is a handy method in Pandas to identify duplicates in a Dataframe (It is called 'duplicated'):

In [11]:
print ('Number of duplicates in kaggle_data: {}'.format(
	sum(kaggle_data.duplicated(subset=['movie_title', 'title_year'], keep=False))))


Number of duplicates in kaggle_data: 241


Now follow the same programming construction shown above to identify and print the number of duplicates in the genres_data and ratings_data dataframes. Note that you will have to use the correct column names for movie and year. 

In [14]:
print ('Number of duplicates in genres_data: {}'.format(
	sum(genres_data.duplicated(subset=['movie', 'year'], keep=False))))

print ('Number of duplicates in ratings_data: {}'.format(
	sum(ratings_data.duplicated(subset=['movie', 'year'], keep=False))))

Number of duplicates in genres_data: 2031322
Number of duplicates in ratings_data: 342815


Dealing with duplicates: There are many strategies for dealing with duplicates. In this case, we are going to assume we can keep the first occurrence and just drop/delete the remaining duplicates, using the drop_duplicates method in the Dataframe class. 

In [16]:
kaggle_data = kaggle_data.drop_duplicates(subset=['movie_title', 'title_year'], keep='first').copy()


Do the same for genres_data and ratings_data

In [17]:
genres_data = genres_data.drop_duplicates(subset=['movie', 'year'], keep='first').copy()
ratings_data = ratings_data.drop_duplicates(subset=['movie', 'year'], keep='first').copy()


Some data integration tasks can be brute force.  In our efforts to integrate the IMDB data with the Kaggle data set, we will be trying to match movie title and year. Ideally we want to make sure the title strings look as similar as possible. There are some simple cleaning steps we can take to try to simplify our integration tasks. These include:



*   Making titles all lower case
*   Getting rid of any commas in all the titles
*   Eliminating quotes
*   Replacing '&' with 'and'
*   Getting rid of question marks

We have started you out. Please finish coding to replace the other characters as described above with additional title.replace lines (you can replace things with '' to get rid of them)










In [18]:
def preprocess_title(title):
	title = title.lower()
	title = title.replace(',', ' ')
	title = title.replace('\"', '')
	title = title.replace('&', 'and')
	title = title.replace('?', '')

	return title.strip()

kaggle_data['norm_movie_title'] = kaggle_data['movie_title'].map(preprocess_title)
genres_data['norm_movie'] = genres_data['movie'].map(preprocess_title)
ratings_data['norm_movie'] = ratings_data['movie'].map(preprocess_title)

In [20]:
kaggle_data.sample(3, random_state=3)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,norm_movie_title,norm_title_year
1714,Color,Sergio Leone,111.0,251.0,0.0,642.0,Burt Young,22000.0,5300000.0,Crime|Drama,Robert De Niro,Once Upon a Time in America,221000,24719,Treat Williams,8.0,1920s|ambiguity|childhood friend|new york|spea...,http://www.imdb.com/title/tt0087843/?ref_=fn_t...,495.0,English,Italy,R,30000000.0,1984.0,683.0,8.4,1.85,12000,once upon a time in america,1984.0
805,Color,Iain Softley,161.0,106.0,34.0,466.0,Stephen Graham,3000.0,17281832.0,Adventure|Family|Fantasy,Brendan Fraser,Inkheart,60232,5329,Jamie Foreman,3.0,book|escape|girl|overalls|villain,http://www.imdb.com/title/tt0494238/?ref_=fn_t...,110.0,English,Germany,PG,60000000.0,2008.0,1000.0,6.1,2.35,0,inkheart,2008.0
684,Color,McG,96.0,131.0,368.0,383.0,Kimberly Williams-Paisley,11000.0,43532294.0,Drama|Sport,Matthew McConaughey,We Are Marshall,46951,12088,Brian Geraghty,0.0,coach|football|football team|marshall universi...,http://www.imdb.com/title/tt0758794/?ref_=fn_t...,175.0,English,USA,PG,65000000.0,2006.0,529.0,7.1,2.35,0,we are marshall,2006.0


What is the data type of the title_year attribute above?  Let's create a little preprocess method that we can apply to the kaggle data title_year attribute to normalize it -- that is store the year as a string and replace null values with '?'. **Fill in the missing code in the method below to convert to a string before running.** 

title_year seems to be a float data type

In [21]:
def preprocess_year(year):
	if pd.isnull(year):
		return '?'
	else:
		return str(int(year))

kaggle_data['norm_title_year'] = kaggle_data['title_year'].map(preprocess_year)
kaggle_data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,norm_movie_title,norm_title_year
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000,avatar,2009
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0,pirates of the caribbean: at world's end,2007
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000,spectre,2015
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000,the dark knight rises,2012
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,NaN,0.0,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0,star wars: episode vii - the force awakens,?


Can you now come up with some code that will display the size of this Dataframe?  How many rows and columns?  In the next exercise we do, we will try to merge and match this Kaggle Data Set with the IMDB data. 


In [26]:
kaggle_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4919 entries, 0 to 5042
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      4900 non-null   object 
 1   director_name              4817 non-null   object 
 2   num_critic_for_reviews     4870 non-null   float64
 3   duration                   4904 non-null   float64
 4   director_facebook_likes    4817 non-null   float64
 5   actor_3_facebook_likes     4896 non-null   float64
 6   actor_2_name               4906 non-null   object 
 7   actor_1_facebook_likes     4912 non-null   float64
 8   gross                      4056 non-null   float64
 9   genres                     4919 non-null   object 
 10  actor_1_name               4912 non-null   object 
 11  movie_title                4919 non-null   object 
 12  num_voted_users            4919 non-null   int64  
 13  cast_total_facebook_likes  4919 non-null   int64

**Prepping the file (to turn into html or pdf) for submission:**

After you have completed all the steps in the notebook above, you will need to run a package called nbconvert to convert your Python Notebook file (.ipynb) to a html file for submission on Canvas. If you have Jupyter Notebook on your personal computer, you will likely have nbconvert as well (and you may be able to convert to pdf). You can download the .ipynb file to your computer from Google colab and run the nbconvert command using 'jupyter nbconvert...' with the rest of the command as shown below. 

The following steps will allow you to convert the file directly here in Google Colab so you won't need any software on your computer.  Follow these steps to convert the file within Google Colab. The process is:


1.   Donwload your .ipynb file to your local computer using the download .ipynb  menu option in the File menu in Colab
2.   Now you need to upload the notebook to your temporary Google data filespace so that your notebook can operate on it. DO NOT UPPLOAD from the FILE menu in colab. Instead, click on the little icon that looks like a file folder on the left side of the colab window. This is your temporary file space. Upload the .ipynb file to that space using the upload icon (with the up arrow) in your files.  

3. Now run the Code block below



In [28]:

! jupyter nbconvert --to html /DS320_cleaning.ipynb

[NbConvertApp] Converting notebook /DS320_cleaning.ipynb to html
[NbConvertApp] Writing 329588 bytes to /DS320_cleaning.html


Refresh your tempoary filespace by clicking on the refresh folder icon. You should see the PDF file there now. Go ahead and download this file by selecting in it and clicking on the three dots on the right sidfe for a download option. 

There are other ways to do this too. You could mount your Google Drive and provide the correct path to the .ipynb file in the command above. 